In [1]:
import numpy as np
import matplotlib.pyplot as plt
import em_stools

In [2]:
dist_ratio = 0.5

In [3]:
Num_sensors1 = 5
Num_emitters1 = 1
sample_size1 = 100
theta1_rad = [0.5] # Направление прибытия (DOA) в радианах
theta1_deg = np.rad2deg(theta1_rad[0]) # Направление прибытия (DOA) в градусах
P_1 = 1 * np.eye(Num_emitters1, dtype=np.float64) # Ковариация шума
Q_1 = 1.1 * np.eye(Num_sensors1, dtype=np.float64) # Ковариация шума
A1 = np.exp(-2j * np.pi * dist_ratio * np.arange(Num_sensors1).reshape(-1,1) * np.sin(theta1_rad)) # Матрица управляющих векторов
# Генерация векторов сигнала, шума и принятого сигнала
S1 = em_stools.gss(Num_emitters1, sample_size1, P_1)
n1 = em_stools.gss(Num_sensors1, sample_size1, Q_1)
X1 = (A1 @ S1.T + n1.T).T

In [4]:
X1_with_mv = em_stools.MCAR(X1, [2,4], [40, 40])

In [ ]:
from numpy.linalg import eig
from scipy.signal import find_peaks

def music_doa(R, num_sources, scan_angles=np.arange(-90, 91, 0.5)):
    L = R.shape[0]
    eigvals, eigvecs = eig(R)
    idx = np.argsort(eigvals)
    eigvals = eigvals[idx]
    eigvecs = eigvecs[:, idx]
    En = eigvecs[:, :-num_sources] 
    P_music = []
    for theta in scan_angles:
        a = np.exp(-1j * 2 * np.pi * 0.5 * np.arange(L) * np.sin(np.deg2rad(theta)))
        a = a.reshape(-1, 1)
        denom = np.conjugate(a.T) @ En @ np.conjugate(En.T) @ a
        P_music.append(1 / np.abs(denom)[0, 0])

    P_music = np.array(P_music)
    P_music_db = 10 * np.log10(P_music / np.max(P_music))
    peaks, _ = find_peaks(P_music_db, distance=5)
    peak_vals = P_music_db[peaks]
    top_peaks = peaks[np.argsort(peak_vals)[-num_sources:]]
    doa_estimates = np.sort(scan_angles[top_peaks])
    return np.deg2rad(doa_estimates)

In [6]:
X1_known = X1_with_mv[~np.isnan(X1_with_mv).any(axis=1)]

In [7]:
X1_known.shape

(37, 5)

In [17]:
R1 = em_stools.complex_cov(X1_known[:6])

In [18]:
music_doa(R1,1)

array([0.41015237])